In [ ]:
import csv
import json
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

# Load pandas
import pandas as pd

# Load numpy
import numpy as np

# Set random seed
np.random.seed(0)

In [2]:
# get language codes
languagecodes = set()

with open('language-codes.csv') as infile:
    for line in infile:
        languagecodes.add(line.strip())

In [3]:
# Y as in input data
y = {}

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

print file_len('final-userdata.csv')

with open('final-userdata.csv') as infile:
    first_line = infile.readline()
    for line in infile:
        #l = json.loads(line.strip().replace('\n',''))
        y.update(json.loads(line.strip().replace('\n','')))


#counter = 0
#with open('final-userdata.csv') as infile:
#    for line in infile:
#        line = json.loads(line)
#        print line 
#        break
#        line = line.split('}{')
#        for l in line:
#            if counter == 0:
#                l = l + '}'
#            elif counter == len(line):
#                l = '{' + l
#            else:
#                l = '{' + l + '}'
#            try:
#                y.update(json.loads(l.strip()))
#            except:
#                print 'FAILED: ' + l
#            counter += 1
#print counter

4119


In [5]:
# X as in predicted data
# get language codes used by the editors ("real language codes")
x = {}

reallanguagecodes = set()

ls = []

userdata = json.load(open('../../compare-labels/userdata.json'))
#print userdata
for k, v in userdata.iteritems():
    k = k.replace('User:', '')
    x[k] = {}
    #print v
    for lang in v:
        level = lang[-2:].replace('-', '')
        if level.lower() == 'n':
            level = 6
        language = lang[:-2]
        ls.append(language)
        if language not in languagecodes:
            continue
        x[k][language] = int(level)
        
for l in languagecodes:
    if l in ls:
        reallanguagecodes.add(l)


In [6]:
# prepare data

dfy = pd.DataFrame(columns=list(languagecodes), index=y.keys())
#print df

for k in y.keys():
    langs = {}
    for l in languagecodes:
        if l in y[k]:
            langs[l] = y[k][l]
        else:
            langs[l] = 0
    dfy.loc[k] = pd.Series(langs)
    
print dfy

                             aln    gv     gu    scn  sr-ec  sr-el    sco  \
Jianhui67                      0    44    722      2      1      1      2   
Zara-arush                     1     1     75      1    247    247      1   
Magnus Manske/uk candidates    1     1      1      6      1      6     21   
Leag                           0     1      1     19      0      1     19   
QuiteUnusual                   0     1      1     19      0      1     35   
Derzno                         1     1      1      1      1      1      1   
Rodguerrer                     1     1      1      1      6      1     69   
Masegand                       1     1      1      2     22      2      2   
Renessaince                    0    44    722      1      1     17     34   
Tohaomg                       65    65     65     65      8     10      1   
Timk70                         0    44    722      1      1     17     34   
Pcm                            1     1      1      2      1      2      2   

In [7]:
# prepare data

dfx = pd.DataFrame(columns=list(languagecodes), index=y.keys())
#print df

for k in y.keys():
    langs = {}
    for l in languagecodes:
        if l in x[k]:
            langs[l] = x[k][l]
        else:
            langs[l] = 0
    dfx.loc[k] = pd.Series(langs)

dfx.columns = [str(col) + '_x' for col in dfx.columns]
print dfx

                             aln_x  gv_x  gu_x  scn_x  sr-ec_x  sr-el_x  \
Jianhui67                        0     0     0      0        0        0   
Zara-arush                       0     0     0      0        0        0   
Magnus Manske/uk candidates      0     0     0      0        0        0   
Leag                             0     0     0      0        0        0   
QuiteUnusual                     0     0     0      0        0        0   
Derzno                           0     0     0      0        0        0   
Rodguerrer                       0     0     0      0        0        0   
Masegand                         0     0     0      0        0        0   
Renessaince                      0     0     0      0        0        0   
Tohaomg                          0     0     0      0        0        0   
Timk70                           0     0     0      0        0        0   
Pcm                              0     0     0      0        0        0   
Achim Raschka            

In [8]:
df = pd.concat([dfy, dfx], axis=1)
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
df.head()

,aln,gv,gu,scn,sr-ec,sr-el,sco,mnw,zh-hk,gd,...,nv_x,ku_x,zh-yue_x,zh-classical_x,bcl_x,bcc_x,lat-vul_x,cps_x,agq_x,is_train
Jianhui67,0,44,722,2,1,1,2,0,3613,1,...,0,0,0,0,0,0,0,0,0,True
Zara-arush,1,1,75,1,247,247,1,0,60,1,...,0,0,0,0,0,0,0,0,0,True
Magnus Manske/uk candidates,1,1,1,6,1,6,21,0,1,6,...,0,0,0,0,0,0,0,0,0,True
Leag,0,1,1,19,0,1,19,0,211,19,...,0,0,0,0,0,0,0,0,0,True
QuiteUnusual,0,1,1,19,0,1,35,0,211,19,...,0,0,0,0,0,0,0,0,0,True


In [9]:
train, test = df[df['is_train']==True], df[df['is_train']==False]
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

('Number of observations in the training data:', 3075)
('Number of observations in the test data:', 1043)


In [10]:
features = df.columns[:281]
features

Index([u'aln', u'gv', u'gu', u'scn', u'sr-ec', u'sr-el', u'sco', u'mnw',
       u'zh-hk', u'gd',
       ...
       u'ce', u'cy', u'sah', u'cs', u'cr', u'cv', u'cu', u'frc', u'ps', u'pt'],
      dtype='object', length=281)

In [11]:
yx = train['en_x'].values

In [17]:


#from sklearn.neighbors import KNeighborsClassifier
#clf = KNeighborsClassifier(n_jobs=2)

clf = RandomForestClassifier(n_jobs=2, random_state=0)

#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
#clf = GaussianProcessClassifier(1.0 * RBF(1.0))

clf.fit(train[features], yx)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [18]:
clf.predict(test[features])

array([2, 2, 4, ..., 0, 4, 3])

In [19]:
d = clf.predict_proba(test[features])
results_pred = []
for a in d:
    results_pred.append(np.argmax(a))

In [20]:
test['en_x'][0:5]
results_pred[0:5]

[2, 2, 4, 6, 2]

In [21]:
list(zip(train[features], clf.feature_importances_))

[('aln', 0.00016476674234899692),
 ('gv', 0.00087136251372480302),
 ('gu', 0.0034085777694250257),
 ('scn', 0.001609058411574845),
 ('sr-ec', 0.0020149552487035501),
 ('sr-el', 0.0015749843942301765),
 ('sco', 0.0031029666679480276),
 ('mnw', 0.0),
 ('zh-hk', 0.0030917071701124202),
 ('gd', 0.0033699253117197544),
 ('ga', 0.002207293604845006),
 ('gn', 0.0003029112866656051),
 ('gl', 0.0065878996427901045),
 ('als', 0.0026253685657254148),
 ('mwv', 0.0),
 ('vep', 0.00022758457460184123),
 ('lki', 0.0),
 ('tw', 0.00038725652513280963),
 ('tt', 0.0025115492331001729),
 ('tr', 0.01437591255835304),
 ('ts', 0.00051769625861838257),
 ('bqi', 0.00036338647380628841),
 ('tn', 0.00030789939054936736),
 ('to', 0.00060905133673477493),
 ('tl', 0.0033012366669223879),
 ('tk', 0.0011786016595734066),
 ('th', 0.0040228997161540136),
 ('ti', 0.00010788902974009405),
 ('tg', 0.0014917243435177089),
 ('lkt', 0.0),
 ('te', 0.0047074664769084593),
 ('mwl', 0.00096968493553408581),
 ('lrc', 3.67555715372

In [22]:
#accuracy = (results_pred. is test['en_x'].values).all(axis=(0,2)).mean()

import sklearn
acc = sklearn.metrics.accuracy_score(np.array(test['en_x'].values), np.array(results_pred))
#clf.predict_proba(test[features])
#print test['en_x'].values
print acc

0.182166826462
